In [22]:
from ast import literal_eval
import itertools

import numpy as np
import pandas as pd

##Set the Columns to show all the data
pd.set_option("display.max_columns", 100)

In [23]:
##Read the raw steam spy data
raw_steamspy_data = pd.read_csv("downloads/steamspy_app_data.csv")
raw_steamspy_data.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,230243,6010,0,"10,000,000 .. 20,000,000",8663,169,154,95,999,999,0,"English, French, German, Italian, Spanish - Sp...",Action,13122,"{'Action': 5470, 'FPS': 4896, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,7032,1065,0,"5,000,000 .. 10,000,000",68,0,20,0,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,68,"{'Action': 763, 'FPS': 327, 'Multiplayer': 277..."
2,30,Day of Defeat,Valve,Valve,NaN,6062,667,0,"5,000,000 .. 10,000,000",619,0,29,0,499,499,0,"English, French, German, Italian, Spanish - Spain",Action,90,"{'FPS': 798, 'World War II': 269, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,2423,514,0,"5,000,000 .. 10,000,000",15,0,13,0,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,3,"{'Action': 635, 'FPS': 150, 'Classic': 115, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,20819,1061,0,"2,000,000 .. 5,000,000",215,0,94,0,499,499,0,"English, French, German, Korean",Action,103,"{'FPS': 915, 'Action': 349, 'Classic': 280, 'S..."


In [24]:
##Print the null counts for each column
raw_steamspy_data.isnull().sum()

appid               0
name                0
developer           0
publisher           0
score_rank         50
positive            0
negative            0
userscore           0
owners              0
average_forever     0
average_2weeks      0
median_forever      0
median_2weeks       0
price               0
initialprice        0
discount            0
languages           0
genre               0
ccu                 0
tags                0
dtype: int64

In [25]:
##Check for null names
raw_steamspy_data[raw_steamspy_data["name"].isnull()]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags


In [26]:
##Check for any missing names
raw_steamspy_data[raw_steamspy_data["name"] == "none"]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags


In [27]:
##Check userscore
raw_steamspy_data["userscore"].value_counts().head()

userscore
0    50
Name: count, dtype: int64

In [28]:
##Drop these columns
drop_cols = [
    "score_rank", # too many missing values
    "userscore", # too little variance (most have 0)
    "genre", "developer", "publisher", "price", "initialprice", "discount", # provided by Steam data
    "average_2weeks", "median_2weeks", "ccu", "owners" # not interested in temporally specific columns
]

In [29]:
##Check the languages row
raw_steamspy_data["languages"].head()

0    English, French, German, Italian, Spanish - Sp...
1    English, French, German, Italian, Spanish - Sp...
2    English, French, German, Italian, Spanish - Spain
3    English, French, German, Italian, Spanish - Sp...
4                      English, French, German, Korean
Name: languages, dtype: object

In [30]:
##Check the tags columns
tags = raw_steamspy_data["tags"]

print(tags[0])
tags.head()

{'Action': 5470, 'FPS': 4896, 'Multiplayer': 3442, 'Shooter': 3392, 'Classic': 2819, 'Team-Based': 1896, 'First-Person': 1736, 'Competitive': 1631, 'Tactical': 1370, "1990's": 1230, 'e-sports': 1215, 'PvP': 907, 'Old School': 806, 'Military': 646, 'Strategy': 630, 'Survival': 312, 'Score Attack': 296, '1980s': 276, 'Assassin': 237, 'Nostalgia': 179}


0    {'Action': 5470, 'FPS': 4896, 'Multiplayer': 3...
1    {'Action': 763, 'FPS': 327, 'Multiplayer': 277...
2    {'FPS': 798, 'World War II': 269, 'Multiplayer...
3    {'Action': 635, 'FPS': 150, 'Classic': 115, 'M...
4    {'FPS': 915, 'Action': 349, 'Classic': 280, 'S...
Name: tags, dtype: object

In [31]:
##Chenge the data in the rows so that it is regocnised by python
eval_row = literal_eval(tags[0])

print(eval_row["Action"])
print(eval_row["FPS"])

5470
4896


In [32]:
parsed_tags = tags.apply(lambda x: literal_eval(x))

cols = set(itertools.chain(*parsed_tags))

print("Number of unique tags:", len(cols))
print("\nFirst few tags:", sorted(list(cols))[:5])

Number of unique tags: 163

First few tags: ['1980s', "1990's", '2.5D', '2D', '3D']


In [33]:
def parseTags(x):

    x = literal_eval(x)
    
    if isinstance(x, dict):##If x is a dictionary 
        return x
    elif isinstance(x, list):##If x is a list
        return {}
    else:
        raise TypeError("Something other than dict or list found")

parsed_tags = tags.apply(parseTags)
        
tag_data = pd.DataFrame()

for col in sorted(cols):
    #Standardise column names
    col_name = col.lower().replace(" ", "_").replace("-", "_").replace("'", "")

    #Check if column in row's dictionary of tags and return that value if it is, or 0 if it isn't
    tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)

tag_data.head()

C:\Users\lolbo\AppData\Local\Temp\ipykernel_10356\4014773965.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)
C:\Users\lolbo\AppData\Local\Temp\ipykernel_10356\4014773965.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)
C:\Users\lolbo\AppData\Local\Temp\ipykernel_10356\4014773965.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

,1980s,1990s,2.5d,2d,3d,3d_platformer,3d_vision,4x,action,action_rpg,action_adventure,addictive,adventure,aliens,alternate_history,animation_&_modeling,anime,arena_shooter,assassin,atmospheric,base_building,based_on_a_novel,benchmark,boomer_shooter,building,bullet_time,cartoon,cartoony,casual,character_customization,cinematic,city_builder,class_based,classic,co_op,collectathon,comedy,competitive,conspiracy,controller,crafting,crime,cult_classic,cute,cyberpunk,dark,dark_comedy,dark_fantasy,dark_humor,demons,...,real_time_tactics,real_time,realistic,remake,replay_value,resource_management,retro,robots,rome,sandbox,sci_fi,science,score_attack,sexual_content,shooter,short,silent_protagonist,simulation,singleplayer,software,space,sports,stealth,story_rich,strategy,stylized,surreal,survival,survival_horror,swordplay,tactical,tanks,team_based,third_person,third_person_shooter,top_down,top_down_shooter,touch_friendly,tower_defense,trading,turn_based,turn_based_strategy,video_production,violent,war,warhammer_40k,world_war_i,world_war_ii,zombies,e_sports
0,276,1230,0,0,0,0,0,0,5470,0,0,0,0,0,0,0,0,0,237,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2819,0,0,0,1631,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,296,0,3392,0,0,0,0,0,0,0,0,0,630,0,0,312,0,0,1370,0,1896,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1215
1,0,157,0,0,0,0,0,0,763,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,195,252,98,0,0,82,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,38,0,0,68,0,0,0,0,0,0,0,221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,201,0,0,0,0,0,0,0,0,0,0,55,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,164,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,83,129,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,22,0,0,0,0,0,0,0,193,0,0,0,43,0,0,0,0,0,15,0,0,0,0,0,45,0,137,0,0,0,0,0,0,0,0,0,0,0,158,0,12,269,0,0
3,0,14,0,0,0,0,0,0,635,0,0,0,0,0,0,0,0,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,115,15,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,20,0,0,0,36,0,0,0,100,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,157,0,0,0,0,0,0,349,0,0,0,135,202,0,0,0,0,0,123,0,0,0,0,0,0,0,0,0,0,0,0,0,280,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37,0,0,0,274,0,0,0,242,0,89,0,253,0,0,0,0,95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
def parseTags(x):
    ##If x is a dictionary join the first three tags together and return it
    x = literal_eval(x)

    if isinstance(x, dict):
        return ";".join(list(x.keys())[:3])
    else:
        return np.nan
    
tags.apply(parseTags).head()

0          Action;FPS;Multiplayer
1          Action;FPS;Multiplayer
2    FPS;World War II;Multiplayer
3              Action;FPS;Classic
4              FPS;Action;Classic
Name: tags, dtype: object

In [35]:
##Review rating = (positive reviews / total treviews) * 10

raw_steamspy_data["total reviews"] = raw_steamspy_data["positive"] + raw_steamspy_data["negative"]

raw_steamspy_data["review_rating"] = (raw_steamspy_data["positive"] / raw_steamspy_data["total reviews"]) * 10

##Rounds the rating to two decimal places

raw_steamspy_data["review_rating"] = raw_steamspy_data["review_rating"].round()

##Convert float to string

raw_steamspy_data["review_rating"] = raw_steamspy_data["review_rating"] .astype(str)

raw_steamspy_data["review_rating"] = raw_steamspy_data["review_rating"] 

raw_steamspy_data["review_rating"].head()

0    10.0
1     9.0
2     9.0
3     8.0
4    10.0
Name: review_rating, dtype: object

In [36]:
def process_tags(df, export =False):##CHANGE THE NAMES OF THIS FUNCTION###
    if export: 
        
        tag_data = df[["appid", "tags"]].copy()
        
        def parse_export_tags(x):
            x = literal_eval(x)

            if isinstance(x, dict):
                return x
            elif isinstance(x, list):
                return {}
            else:
                raise TypeError("Something other than dict or list found")

        tag_data["tags"] = tag_data["tags"].apply(parse_export_tags)

        cols = set(itertools.chain(*tag_data["tags"]))

        for col in sorted(cols):
            col_name = col.lower().replace(" ", "_").replace("-", "_").replace("'", "")

            tag_data[col_name] = tag_data["tags"].apply(lambda x: x[col] if col in x.keys() else 0)

        tag_data = tag_data.drop("tags", axis=1)

        tag_data.to_csv("exports/steamspy_tag_data.csv", index = False)
        print("Exported tag data to 'exports/steamspy_tag_data.csv'")
        
        
    def parse_tags(x):
        x = literal_eval(x)
        
        if isinstance(x, dict):
            return ";".join(list(x.keys())[:3])
        else:
            return np.nan
    
    df["tags"] = df["tags"].apply(parse_tags)
    
    # rows with null tags seem to be superseded by newer release, so remove (e.g. dead island)
    df = df[df["tags"].notnull()]
    
    return df


def process(df):
    df = df.copy()
    
    # handle missing values
    df = df[(df["name"].notnull()) & (df["name"] != "none")]
    df = df[df["developer"].notnull()]
    df = df[df["languages"].notnull()]
    df = df[df["price"].notnull()]
    
    # remove unwanted columns
    df = df.drop([
        "genre", "developer", "publisher", "score_rank", "userscore", "average_2weeks",
        "median_2weeks", "price", "initialprice", "discount", "ccu"
    ], axis=1)
    
    # keep top tags, exporting full tag data to file
    df = process_tags(df, export=True)
    
    # reformat owners column
    df["owners"] = df["owners"].str.replace(",", "").str.replace(" .. ", "-")
    
    return df


steamspy_data = process(raw_steamspy_data)
steamspy_data.head()

Exported tag data to 'exports/steamspy_tag_data.csv'


C:\Users\lolbo\AppData\Local\Temp\ipykernel_10356\3852328670.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = tag_data["tags"].apply(lambda x: x[col] if col in x.keys() else 0)
C:\Users\lolbo\AppData\Local\Temp\ipykernel_10356\3852328670.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = tag_data["tags"].apply(lambda x: x[col] if col in x.keys() else 0)
C:\Users\lolbo\AppData\Local\Temp\ipykernel_10356\3852328670.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is u

,appid,name,positive,negative,owners,average_forever,median_forever,languages,tags,total reviews,review_rating
0,10,Counter-Strike,230243,6010,10000000-20000000,8663,154,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer,236253,10.0
1,20,Team Fortress Classic,7032,1065,5000000-10000000,68,20,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer,8097,9.0
2,30,Day of Defeat,6062,667,5000000-10000000,619,29,"English, French, German, Italian, Spanish - Spain",FPS;World War II;Multiplayer,6729,9.0
3,40,Deathmatch Classic,2423,514,5000000-10000000,15,13,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Classic,2937,8.0
4,50,Half-Life: Opposing Force,20819,1061,2000000-5000000,215,94,"English, French, German, Korean",FPS;Action;Classic,21880,10.0


In [37]:
##Inspect tag data
pd.read_csv("exports/steamspy_tag_data.csv").head()

,appid,1980s,1990s,2.5d,2d,3d,3d_platformer,3d_vision,4x,action,action_rpg,action_adventure,addictive,adventure,aliens,alternate_history,animation_&_modeling,anime,arena_shooter,assassin,atmospheric,base_building,based_on_a_novel,benchmark,boomer_shooter,building,bullet_time,cartoon,cartoony,casual,character_customization,cinematic,city_builder,class_based,classic,co_op,collectathon,comedy,competitive,conspiracy,controller,crafting,crime,cult_classic,cute,cyberpunk,dark,dark_comedy,dark_fantasy,dark_humor,...,real_time_tactics,real_time,realistic,remake,replay_value,resource_management,retro,robots,rome,sandbox,sci_fi,science,score_attack,sexual_content,shooter,short,silent_protagonist,simulation,singleplayer,software,space,sports,stealth,story_rich,strategy,stylized,surreal,survival,survival_horror,swordplay,tactical,tanks,team_based,third_person,third_person_shooter,top_down,top_down_shooter,touch_friendly,tower_defense,trading,turn_based,turn_based_strategy,video_production,violent,war,warhammer_40k,world_war_i,world_war_ii,zombies,e_sports
0,10,276,1230,0,0,0,0,0,0,5470,0,0,0,0,0,0,0,0,0,237,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2819,0,0,0,1631,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,296,0,3392,0,0,0,0,0,0,0,0,0,630,0,0,312,0,0,1370,0,1896,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1215
1,20,0,157,0,0,0,0,0,0,763,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,195,252,98,0,0,82,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,38,0,0,68,0,0,0,0,0,0,0,221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,201,0,0,0,0,0,0,0,0,0,0,55,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,164,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,83,129,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,22,0,0,0,0,0,0,0,193,0,0,0,43,0,0,0,0,0,15,0,0,0,0,0,45,0,137,0,0,0,0,0,0,0,0,0,0,0,158,0,12,269,0,0
3,40,0,14,0,0,0,0,0,0,635,0,0,0,0,0,0,0,0,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,115,15,0,0,29,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,20,0,0,0,36,0,0,0,100,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,0,157,0,0,0,0,0,0,349,0,0,0,135,202,0,0,0,0,0,123,0,0,0,0,0,0,0,0,0,0,0,0,0,280,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37,0,0,0,274,0,0,0,242,0,89,0,253,0,0,0,0,95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
##Final check for nill data
steamspy_data.isnull().sum()

appid              0
name               0
positive           0
negative           0
owners             0
average_forever    0
median_forever     0
languages          0
tags               0
total reviews      0
review_rating      0
dtype: int64

In [39]:
##Final export of steam spy data
steamspy_data.to_csv("exports/steamspy_clean.csv", index = False)

In [40]:
##Read the cleaned steam data 
steam_data = pd.read_csv("exports/steam_data_clean.csv")
    
merged = steam_data.merge(steamspy_data, left_on  = "steam_appid", right_on = "appid", suffixes = ("", "_steamspy"))
merged.head()

,name,steam_appid,required_age,header_image,website,pc_requirements,mac_requirements,linux_requirements,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors,price,english,developer,publisher,appid,name_steamspy,positive,negative,owners,average_forever,median_forever,languages,tags,total reviews,review_rating
0,Team Fortress Classic,20,0,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 6000},1999-04-01,"{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",4.29,1,Valve,Valve,20,Team Fortress Classic,7032,1065,5000000-10000000,68,20,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer,8097,9.0
1,Day of Defeat,30,0,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,"{'score': 79, 'url': 'https://www.metacritic.c...",Multi-player;Valve Anti-Cheat enabled;Family S...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 4008},2003-05-01,"{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve,30,Day of Defeat,6062,667,5000000-10000000,619,29,"English, French, German, Italian, Spanish - Spain",FPS;World War II;Multiplayer,6729,9.0
2,Deathmatch Classic,40,0,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 2104},2001-06-01,"{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve,40,Deathmatch Classic,2423,514,5000000-10000000,15,13,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Classic,2937,8.0
3,Half-Life: Opposing Force,50,0,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Single-player;Multi-player;Valve Anti-Cheat en...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 18753},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Gearbox Software,Valve,50,Half-Life: Opposing Force,20819,1061,2000000-5000000,215,94,"English, French, German, Korean",FPS;Action;Classic,21880,10.0
4,Ricochet,60,0,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Valve Anti-Cheat e...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 3973},2000-11-01,"{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve,60,Ricochet,4617,993,5000000-10000000,7,2,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer,5610,8.0


In [41]:
##Remove overlapping columns
steam_clean = merged.drop(["name_steamspy", "languages", "steam_appid"], axis = 1)

##Reindex to reorder columns
steam_clean = steam_clean[[
    "appid",
    "name",
    "release_date",
    "english",
    "developer",
    "publisher",
    "platforms",
    "required_age",
    "categories",
    "genres",
    "tags",
    "positive",
    "negative",
    "total reviews",
    "review_rating",
    "average_forever",
    "median_forever",
    "owners",
    "price"
]]

steam_clean = steam_clean.rename({
    "positive": "positive_ratings",
    "negative": "negative_ratings",
    "average_forever": "average_playtime",
    "median_forever": "median_playtime"
}, axis = 1)

steam_clean.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,tags,positive_ratings,negative_ratings,total reviews,review_rating,average_playtime,median_playtime,owners,price
0,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Multiplayer,7032,1065,8097,9.0,68,20,5000000-10000000,4.29
1,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled;Family S...,Action,FPS;World War II;Multiplayer,6062,667,6729,9.0,619,29,5000000-10000000,4.29
2,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,Action;FPS;Classic,2423,514,2937,8.0,15,13,5000000-10000000,4.29
3,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Classic,20819,1061,21880,10.0,215,94,2000000-5000000,4.29
4,60,Ricochet,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;PvP;Online PvP;Valve Anti-Cheat e...,Action,Action;FPS;Multiplayer,4617,993,5610,8.0,7,2,5000000-10000000,4.29


In [42]:
##Export clean dataset
steam_clean.to_csv("FinalData/steam_clean.csv", index = False)